In [172]:
import os, pandas as pd, numpy as np

In [173]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

In [174]:
import matplotlib.pyplot as plt


In [175]:
loans = pd.read_csv('LoanStats3a.csv', skiprows=1, low_memory=False)
loans.head()


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


Data Cleaning 

In [176]:
loans.shape

(42540, 145)

In [177]:
# explore missing values
loans.isnull().sum()

id                                            42537
member_id                                     42540
loan_amnt                                         5
funded_amnt                                       5
funded_amnt_inv                                   5
term                                              5
int_rate                                          5
installment                                       5
grade                                             5
sub_grade                                         5
emp_title                                      2631
emp_length                                     1117
home_ownership                                    5
annual_inc                                        9
verification_status                               5
issue_d                                           5
loan_status                                       5
pymnt_plan                                        5
url                                           42540
desc        

In [178]:
# drop columns with more than 50% of missing values
half = len(loans) / 2
loans = loans.dropna(thresh=half, axis=1)
loans.isnull().sum()

loan_amnt                         5
funded_amnt                       5
funded_amnt_inv                   5
term                              5
int_rate                          5
installment                       5
grade                             5
sub_grade                         5
emp_title                      2631
emp_length                     1117
home_ownership                    5
annual_inc                        9
verification_status               5
issue_d                           5
loan_status                       5
pymnt_plan                        5
desc                          13298
purpose                           5
title                            18
zip_code                          5
addr_state                        5
dti                               5
delinq_2yrs                      34
earliest_cr_line                 34
inq_last_6mths                   34
open_acc                         34
pub_rec                          34
revol_bal                   

In [179]:
# drop columns that leak information from the future (information after the loan is already started to be funded)
loans = loans.drop(['funded_amnt','funded_amnt_inv','issue_d','out_prncp','out_prncp_inv','total_pymnt',
                    'total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries',
                    'collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','disbursement_method'], axis=1)

# drop debt_settlement_flag column since there is no info on it in the data dictionary
loans = loans.drop('debt_settlement_flag', axis=1)

In [180]:
# drop columns that contain redundant information
# grade and sub_grade are assigned grade based on the borrower's interest rate, therefore redundant as int_rate
# zip_code is redundant with addr_state as it only show first 3 digits
loans = loans.drop(['grade','sub_grade','zip_code'], axis=1)
loans.head()

,loan_amnt,term,int_rate,installment,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,...,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,hardship_flag
0,5000.0,36 months,10.65%,162.87,NaN,10+ years,RENT,24000.0,Verified,Fully Paid,...,Sep-18,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N
1,2500.0,60 months,15.27%,59.83,Ryder,< 1 year,RENT,30000.0,Source Verified,Charged Off,...,Oct-16,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N
2,2400.0,36 months,15.96%,84.33,NaN,10+ years,RENT,12252.0,Not Verified,Fully Paid,...,Jun-17,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N
3,10000.0,36 months,13.49%,339.31,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Fully Paid,...,Apr-16,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N
4,3000.0,60 months,12.69%,67.79,University Medical Group,1 year,RENT,80000.0,Source Verified,Fully Paid,...,Apr-18,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N


In [181]:
# explore number of unique values in each column
unique_values = loans.apply(pd.Series.nunique,axis=0)
unique_values

loan_amnt                       898
term                              2
int_rate                        394
installment                   16459
emp_title                     30658
emp_length                       11
home_ownership                    5
annual_inc                     5597
verification_status               3
loan_status                       4
pymnt_plan                        1
desc                          28963
purpose                          14
title                         21256
addr_state                       50
dti                            2894
delinq_2yrs                      12
earliest_cr_line                530
inq_last_6mths                   28
open_acc                         44
pub_rec                           6
revol_bal                     22709
revol_util                     1119
total_acc                        83
initial_list_status               1
last_credit_pull_d              134
collections_12_mths_ex_med        1
policy_code                 

In [182]:
# drop columns that contain only one unique value
cols_with_one_value = unique_values[unique_values == 1]
loans = loans.drop(cols_with_one_value.index, axis=1)
loans.head()

,loan_amnt,term,int_rate,installment,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,NaN,10+ years,RENT,24000.0,Verified,Fully Paid,...,3.0,0.0,13648.0,83.70%,9.0,Sep-18,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,Ryder,< 1 year,RENT,30000.0,Source Verified,Charged Off,...,3.0,0.0,1687.0,9.40%,4.0,Oct-16,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,NaN,10+ years,RENT,12252.0,Not Verified,Fully Paid,...,2.0,0.0,2956.0,98.50%,10.0,Jun-17,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Fully Paid,...,10.0,0.0,5598.0,21%,37.0,Apr-16,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,University Medical Group,1 year,RENT,80000.0,Source Verified,Fully Paid,...,15.0,0.0,27783.0,53.90%,38.0,Apr-18,0.0,0.0,0.0,0.0


In [183]:
# explore text columns
loans.select_dtypes(include='object').apply(pd.Series.nunique,axis=0)

term                       2
int_rate                 394
emp_title              30658
emp_length                11
home_ownership             5
verification_status        3
loan_status                4
desc                   28963
purpose                   14
title                  21256
addr_state                50
earliest_cr_line         530
revol_util              1119
last_credit_pull_d       134
dtype: int64

In [184]:
# drop text columns emp_title, desc, and title since they have too many unique categorical values and too hard to process
loans = loans.drop(['emp_title','desc','title', 'addr_state','earliest_cr_line','last_credit_pull_d'],axis=1)
loans.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,...,1.0,3.0,0.0,13648.0,83.70%,9.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,...,5.0,3.0,0.0,1687.0,9.40%,4.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,...,2.0,2.0,0.0,2956.0,98.50%,10.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,...,1.0,10.0,0.0,5598.0,21%,37.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Fully Paid,other,...,0.0,15.0,0.0,27783.0,53.90%,38.0,0.0,0.0,0.0,0.0


 EDA & Feature Preparation

In [185]:
# explore target column
loans['loan_status'].value_counts()

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

In [186]:
# remove loans that are not Fully Paid or Charged Off, sicne we are only interested in those two final outcomes
loans = loans[(loans.loan_status == 'Fully Paid') | (loans.loan_status == 'Charged Off')]

In [187]:
# transform the two labels to numerical values
loans = loans.replace('Fully Paid', 1)
loans = loans.replace('Charged Off', 0)
loans['loan_status'].value_counts()

1    34116
0     5670
Name: loan_status, dtype: int64

Obviously this is a case of imbalanced class problem, which we need to use some approach to handle this so that the model doesn't show bias when making predictions.

In [188]:
# handle missing values with the remaining columns
loans.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1078
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
dti                        0
delinq_2yrs                0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
acc_now_delinq             0
delinq_amnt                0
pub_rec_bankruptcies     697
tax_liens                 39
dtype: int64

In [189]:
loans['emp_length'].value_counts(dropna=False)

10+ years    8899
< 1 year     4590
2 years      4394
3 years      4098
4 years      3444
5 years      3286
1 year       3247
6 years      2231
7 years      1775
8 years      1485
9 years      1259
NaN          1078
Name: emp_length, dtype: int64

In [190]:
# convert emp_length to numeric columns, assuming NaN is 0 year experience
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.replace(mapping_dict)


In [191]:
# drop pub_rec_bankruptcies column and for other columns with null values, remove the rows with null values
loans = loans.drop('pub_rec_bankruptcies', axis=1)
loans = loans.dropna(axis=0)
loans.isnull().sum()

loan_amnt              0
term                   0
int_rate               0
installment            0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
loan_status            0
purpose                0
dti                    0
delinq_2yrs            0
inq_last_6mths         0
open_acc               0
pub_rec                0
revol_bal              0
revol_util             0
total_acc              0
acc_now_delinq         0
delinq_amnt            0
tax_liens              0
dtype: int64

In [192]:
# explore remaining text columns
loans.select_dtypes(include='object').apply(pd.Series.nunique,axis=0)

term                      2
int_rate                371
home_ownership            5
verification_status       3
purpose                  14
revol_util             1087
dtype: int64

In [193]:
# convert the int_rate column and revol_util column to numeric columns
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype('float')

In [194]:
# create dummy variables for the remaining categorical variables in the dataframe
dummy_df = pd.get_dummies(loans[['home_ownership','verification_status','purpose','term']])
loans = pd.concat([loans,dummy_df],axis=1)
loans = loans.drop(['home_ownership','verification_status','purpose','term'],axis=1)

In [198]:
loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10.0,24000.0,1,27.65,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0.0,30000.0,0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10.0,12252.0,1,8.72,0.0,2.0,2.0,...,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10.0,49200.0,1,20.00,0.0,1.0,10.0,...,0,0,0,1,0,0,0,0,1,0
4,3000.0,12.69,67.79,1.0,80000.0,1,17.94,0.0,0.0,15.0,...,0,0,0,1,0,0,0,0,0,1


Modeling 

Set up error metric
Due to imbalanced class, we want to use precision & recall to measure performance instead of accuracy. From the investor's standpoint, we want to accurately identify those who can't pay off the loans (negatives), so we want keep false positive rate as low as possible and also optimize for recall. 


In [199]:
# set up error metric 
def error_metric(predictions):
    tp_filter = (predictions==1) & (loans['loan_status']==1)
    tp = float(len(predictions[tp_filter]))
    
    fn_filter = (predictions==0) & (loans['loan_status']==1)
    fn = float(len(predictions[fn_filter]))
    
    fp_filter = (predictions==1) & (loans['loan_status']==0)
    fp = float(len(predictions[fp_filter]))
    
    tn_filter = (predictions==0) & (loans['loan_status']==0)
    tn = float(len(predictions[tn_filter]))
    
    recall = tp / (tp + fn)
    fpr = fp / (fp + tn)
    
    return fpr, recall

In [202]:
# Logistics Regression
features = loans.drop('loan_status', axis=1)
target = loans['loan_status']

lr = LogisticRegression()
predictions = cross_val_predict(lr,features,target, cv=10)
predictions = pd.Series(predictions)

print(error_metric(predictions))

(0.9992357661444402, 0.9993801716924412)


In [203]:
# penalize the classifier
lr = LogisticRegression(class_weight='balanced')
predictions = cross_val_predict(lr,features,target, cv=10)
predictions = pd.Series(predictions)

print(error_metric(predictions))

(0.6085212074894918, 0.621501843989215)


In [205]:
# manually penalize the classifier
penalty = {0:10, 1:1}
lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr,features,target,cv=10)
predictions = pd.Series(predictions)

print(error_metric(predictions))

(0.21188383645395492, 0.20875817398580593)
